In [1]:
print("all ok")

all ok


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_groq import ChatGroq

In [4]:
llm=ChatGroq(model_name="deepseek-r1-distill-llama-70b")

In [5]:
llm.invoke("What is the capital of France?")

AIMessage(content='<think>\n\n</think>\n\nThe capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 10, 'total_tokens': 22, 'completion_time': 0.046671722, 'prompt_time': 0.000196335, 'queue_time': 0.273293544, 'total_time': 0.046868057}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_e98d30d035', 'finish_reason': 'stop', 'logprobs': None}, id='run--1cbdd6b6-b262-4ebe-9839-46bdabed441c-0', usage_metadata={'input_tokens': 10, 'output_tokens': 12, 'total_tokens': 22})

In [6]:
from langchain_core.tools import tool

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [8]:
@tool
def multiply(x: int, y: int) -> int:
    """Multiplies two numbers."""
    return x * y

In [9]:
multiply({"x":2, "y":3})

C:\Users\sunny\AppData\Local\Temp\ipykernel_87612\4161870413.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  multiply({"x":2, "y":3})


6

In [10]:
@tool
def search(query: str):
    """search the web for a query and return the results"""
    tavily=TavilySearchResults()
    result=tavily.invoke(query)
    return f"Result for {query} is: \n{result}"
    

In [11]:
print(search.invoke({"query":"What is the capital of France?"}))

Result for What is the capital of France? is: 
[{'title': 'What is the Capital of France? - Mappr', 'url': 'https://www.mappr.co/capital-cities/france/', 'content': 'mappr logo\nmappr logo\n\nHome » Capital Cities » What is the Capital of France?\n\n# What is the Capital of France?\n\nParis is the capital of France. Also known as the “City of Light,” is not only the capital but also the most populous city of France. Paris is renowned globally for its art, fashion, gastronomy, and culture.\n\nA map showing France in yellow with surrounding countries in white, major cities marked, and Paris highlighted with a black dot. [...] However, Paris only became the official capital of France during the reign of Clovis I, in the late 5th and early 6th century. Before Clovis, the Merovingian kings did not rule from a fixed capital, but Clovis established his court in Paris, effectively making it the capital. Since then, with some interruptions, Paris has remained the capital of France. [...] The Pa

In [12]:
tools = [multiply, search]

In [13]:
tools

[StructuredTool(name='multiply', description='Multiplies two numbers.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x0000025AB68CB5B0>),
 StructuredTool(name='search', description='search the web for a query and return the results', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x0000025AB68C8AF0>)]

In [14]:
llm_with_tools=llm.bind_tools(tools)

In [16]:
result=llm_with_tools.invoke("what is current gdp of india?")

In [17]:
result.content

''

In [ ]:
result.tool_calls

[{'name': 'search',
  'args': {'query': 'current GDP of India'},
  'id': '9h9hp280y',
  'type': 'tool_call'}]

In [24]:
result.tool_calls[0]["name"]

'search'

In [25]:
result.tool_calls[0]["args"]

{'query': 'current GDP of India'}

In [26]:
type(result.tool_calls[0]["args"])

dict

In [19]:
tool_mapping={tool.name:tool for tool in tools}

In [20]:
tool_mapping

{'multiply': StructuredTool(name='multiply', description='Multiplies two numbers.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x0000025AB68CB5B0>),
 'search': StructuredTool(name='search', description='search the web for a query and return the results', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x0000025AB68C8AF0>)}

In [21]:
tool_mapping["search"]

StructuredTool(name='search', description='search the web for a query and return the results', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x0000025AB68C8AF0>)

In [ ]:
#manually i am passing here
# tool_mapping["search"].invoke({"query":"What is the capital of india?"})

'Result for What is the capital of india? is: \n[{\'title\': \'What is the capital of India? States and union territories explained.\', \'url\': \'https://www.usatoday.com/story/news/world/2023/05/24/what-is-the-capital-of-india/70195720007/\', \'content\': \'Want to learn more about the soon-to-be most populous country? Here’s some interesting information about how India is organized.\\n\\n## What is the capital of India?\\n\\nThe capital of India is New Delhi, located in the north-central part of the country to the west of the Yamuna River.\\n\\nCalcutta (now Kolkata, the capital of West Bengal) was the country’s capital until 1911 when King George V declared Delhi the new capital and construction of New Delhi began. [...] When the national government achieved independence in 1947, New Delhi became the capital.\\n\\nMumbai, the state capital of Maharashtra, is often considered the financial capital of India because of its role in the national and international economy.\\n\\n## How ma

In [ ]:

tool_mapping[result.tool_calls[0]["name"]].invoke(result.tool_calls[0]["args"])

'Result for current GDP of India is: \n[{\'title\': \'India - GDP, PPP (current International $) - 2025 Data 2026 Forecast ...\', \'url\': \'https://tradingeconomics.com/india/gdp-ppp-us-dollar-wb-data.html\', \'content\': \'GDP, PPP (current international $) in India was reported at 14619765554295 USD in 2023, according to the World Bank collection of development indicators, compiled from officially recognized sources. India - GDP, PPP (current international $) - actual values, historical data, forecasts and projections were sourced from the World Bank on June of 2025.\', \'score\': 0.8800674}, {\'title\': "India To Overtake Japan As World\'s 4th Largest Economy In 2025: IMF ...", \'url\': \'https://pwonlyias.com/current-affairs/indias-economic-growth/\', \'content\': "India is projected to become the world\'s fourth-largest economy in 2025 by nominal GDP, surpassing Japan, according to IMF estimates. Current Status of Indian Economy According to the International Monetary Fund (IMF),